# Conversion from Tensorflow to Onnx to C++

This notebook lays out the process of converting trained neural networks to C++ representations using SOFIE. I ran into a few difficulties, though, which I'll discuss below.

## Tensorflow to Onnx

In the KKTrain folder, the trained models have already been saved in h5 format. The code below converts the saved .h5 file into a .pb file (Tensorflow's Saved Model format). This approach worked on both NERSC and my local machine. However, this does not work on flavor02. I'm not sure if it's the tensorflow installation or another issue. (The error that I receive is a TypeError.)

In [1]:
import tensorflow as tf

!mkdir -p "saved_model"

model = tf.keras.models.load_model('TrainBkg.h5', compile=False)
tf.saved_model.save(model, 'saved_model/my_saved_model')

keras.engine.functional.Functional

It's also possible to insert this without saving as an .h5 file first. Use the following code after training the model, where `model` is the neural network:

In [ ]:
!mkdir -p "saved_model"

tf.saved_model.save(model, 'saved_model/my_saved_model')

Install `pip install -U tf2onnx`. In the command line, while in the directory `KKTrain`, use the following command:

In [ ]:
python -m tf2onnx.convert --saved-model 'saved_model/my_saved_model' --output model.onnx

## Onnx to C++
Assumes that ROOT has been built from the source using the following commands:

In [ ]:
git clone git@github.com:root-project/root.git

cd root

mkdir root_install root_build

cd root_build

cmake -DCMAKE_INSTALL_PREFIX=../root_install ../ -Dtmva-sofie=ON -Dtmva-pymva=On -DPython3_EXECUTABLE=/global/homes/j/jnzhao3/.conda/envs/train/bin/python3 -DProtobuf_INCLUDE_DIR=~/.local/
cmake --build . --target install -j2

source <installdir>/bin/thisroot.sh

Here, I've used the path to my own Python executable. If on another machine, use the following commands to find the path:

In [ ]:
python3

import sys
print(sys.executable)

The current issue: `cmake -DCMAKE_INSTALL_PREFIX=../root_install ../ -Dtmva-sofie=ON -Dtmva-pymva=On -DPython3_EXECUTABLE=/global/homes/j/jnzhao3/.conda/envs/train/bin/python3 -DProtobuf_INCLUDE_DIR=~/.local/` does not compile. I think the issue has to do with NERSC's Cray compiler: https://docs.nersc.gov/development/compilers/wrappers/

In the ROOT command line, run the following commands:

In [ ]:
using namespace TMVA::Experimental;
SOFIE::RModelParser_ONNX parser;
SOFIE::RModel model = parser.Parse(“./model.onnx”);
model.Generate();
model.OutputGenerated(“./output.hxx”);

Unfortunately, I ran into errors while installing ROOT from the source on my local device. Because of the issues with converting to Saved Model format on flavor02, I was not able to proceed until this point. Hence, I don't have completed models to show. However, this works just fine on NERSC (where I unfortunately forgot to save my work).